In [74]:
import gensim
import json
import nltk
import re
import unicodedata
from gensim import corpora, models, similarities

In [75]:
with open('result.json') as fp:
    all_results = json.load(fp)

In [76]:
all_results.keys()

documents = []
documents_titles = all_results.keys()
for d in documents_titles:
    text = " ".join(f for n in all_results[d]['depoentes_names'] for f in all_results[d]['text'][n]['falas'])
    documents.append(text)

In [132]:
def rem_accents(text):
    if type(text) == str:
        text = unicode(text, 'utf8')
    return unicodedata.normalize('NFD', text).encode('ascii', 'ignore') #remove accents

# remove common words and tokenize
stoplist = {rem_accents(x) for x in nltk.corpus.stopwords.words('portuguese')} | \
{"sr", "senhor", "deputado", "presidente", "vou", "deputado", "pergunta", "silencio", "permanecer", "permanecerei", "acho", "falei",
 "entao", "processo", "manter", "inclusive", "questao", "momento", "falar", "seguinte", "porque", "aqui", "ser",
 "dizer", "sobre", "ter", "sim", "pode", "fazer", "sei", "la", "ai", "gente", "vai", "relacao", "posso", "todos", "dentro", "tudo", "assim",
"nunca", "parte", "hoje", "agora", "nesse", "excelentissimo", "ilustrissima", "ilustrissimo", "excelencia"} |\
{"petrobras", "empresa", "empresas", "area"}

texts = [[word for word in re.split(r'\W+', document.lower()) if word and word not in stoplist]
         for document in documents]

# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1]
         for text in texts]

from pprint import pprint   # pretty-printer
#pprint(texts)

dictionary = corpora.Dictionary(texts)
dictionary.save('/tmp/deerwester.dict') # store the dictionary, for future reference
print(dictionary)

corpus = [dictionary.doc2bow(text) for text in texts]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
corpora.MmCorpus.serialize('/tmp/deerwester.mm', corpus_tfidf) # store to disk, for later use
#print(corpus)

Dictionary(11524 unique tokens: [u'anulando', u'estacao', u'inversao', u'vivemos', u'bloqueou']...)


In [133]:
import logging, gensim, bz2

lda = gensim.models.ldamodel.LdaModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=10)
for i,t in enumerate(lda.print_topics(num_words=10)):
    print("{} {}\n".format(i,t))

0 0.001*calado + 0.001*impoe + 0.001*triunfo + 0.000*recorro + 0.000*excelencias + 0.000*mentiroso + 0.000*petroquimica + 0.000*exercer + 0.000*banco + 0.000*fechada

1 0.000*petros + 0.000*refinaria + 0.000*shinko + 0.000*deliberativo + 0.000*teste + 0.000*eminente + 0.000*dedurar + 0.000*modelo + 0.000*supremo + 0.000*companhia

2 0.001*janene + 0.001*recomendacao + 0.000*sete + 0.000*advogados + 0.000*financas + 0.000*pt + 0.000*central + 0.000*diretorio + 0.000*seguindo + 0.000*partido

3 0.001*sondas + 0.001*schahin + 0.000*sete + 0.000*petros + 0.000*refinaria + 0.000*portanto + 0.000*companhias + 0.000*setal + 0.000*exercer + 0.000*diretoria

4 0.001*sap + 0.001*camargo + 0.000*correa + 0.000*clientes + 0.000*vendemos + 0.000*softwares + 0.000*inexigibilidade + 0.000*dalton + 0.000*solucoes + 0.000*contratos

5 0.000*bravo + 0.000*leniencia + 0.000*controladoria + 0.000*737 + 0.000*combate + 0.000*predio + 0.000*responsabilizacao + 0.000*sindicancia + 0.000*repressao + 0.000*ins

In [129]:
len(documents_titles)

46

In [131]:
for i in range(len(documents_titles)):
    print("{}\t {}".format(documents_titles[i], lda[corpus[i]]))

15.07.2015- Jose Cardozo-Ministro Justica.pdf	 [(5, 0.013160699661322422), (6, 0.42080441960511528), (7, 0.024373930926144184), (9, 0.54073393152963622)]
16.06.2015-Sete Participacoes.pdf	 [(3, 0.42557346759989206), (6, 0.30737259882716444), (7, 0.26485049858259913)]
16.07.2015- Ministro CGU, e AGU.pdf	 [(6, 0.99779332167654311)]
31.08.2015- Andrade Gutierrez.pdf	 [(0, 0.95504656558505852), (5, 0.014352604253282582), (7, 0.027708063198691145)]
21.05.2015- Vice presidente Engevix.pdf	 [(0, 0.027617232217552138), (4, 0.73290219338501239), (7, 0.019233098047467461), (9, 0.21841779095382061)]
18.08.2015-Doleiros.pdf	 [(4, 0.36006131879674441), (6, 0.60329870640021166), (7, 0.036453119940120501)]
14.10.2015- Presidente Petrobras Bendine.pdf	 [(0, 0.15608513195808094), (4, 0.012797913967698834), (6, 0.73945534381625011), (7, 0.084808701491887545)]
02.09.2015- Publicitario, dono Toyo Setal, etc.pdf	 [(0, 0.026369972254677049), (6, 0.96267355932205145), (7, 0.010435174900120176)]
27.05.2015-Gr